<a href="https://colab.research.google.com/github/jwa0301/Personal-study/blob/main/Transformers_101/Transformers_17_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2024-01-16 완료

# 4장. GPT를 활용한 작문

# 17

In [1]:
# 런타임 3분 소요
!pip install transformers sentencepiece
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# 모델 및 토크나이저 불러오기
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

#18

In [2]:
# 토크나이징을 통한 인코딩(입력문이 하나인 경우)
input = tokenizer.encode("I evaluated the performance of GPT-Neo developed by OpenAI.", return_tensors="pt")

# 첫번째 인코딩 결과(결과적으로 하나의 입력문장)를 확인
print(input[0])

# 첫번째 인코딩 결과를 디코딩
print(tokenizer.decode(input[0]))


tensor([   40, 16726,   262,  2854,   286,   402, 11571,    12,  8199,    78,
         4166,   416,  4946, 20185,    13])
I evaluated the performance of GPT-Neo developed by OpenAI.


In [18]:
# 토크나이징을 통한 인코딩(입력문이 복수인 경우)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# max_len을 지정하지 않을 경우 알아서 max_len에 맞게 padding, truncation 진행
input = tokenizer.batch_encode_plus(["I evaluated the performance of GPT-Neo developed by OpenAI.","I evaluated the performance of GPT developed by OpenAI."], padding=True, truncation=True, return_tensors="pt")

# 인코딩 결과 확인
print(input['input_ids'])

# 디코딩
# 패드 토큰은 마지막 50257로 지정된 것을 볼 수 있음
print([tokenizer.decode(input['input_ids'][i]) for i in range(len(input['input_ids']))])



tensor([[   40, 16726,   262,  2854,   286,   402, 11571,    12,  8199,    78,
          4166,   416,  4946, 20185,    13],
        [   40, 16726,   262,  2854,   286,   402, 11571,  4166,   416,  4946,
         20185,    13, 50257, 50257, 50257]])
['I evaluated the performance of GPT-Neo developed by OpenAI.', 'I evaluated the performance of GPT developed by OpenAI.[PAD][PAD][PAD]']


# 19

In [19]:
# 토크나이징
input = tokenizer.batch_encode_plus(["I evaluated the performance of GPT2 developed by OpenAI.", "Vaccine for new coronavirus in the UK","3.1415926535"], max_length=5, truncation=True, padding=True, return_tensors="pt")

In [20]:
# 인코딩 결과 확인
input['input_ids']

tensor([[   40, 16726,   262,  2854,   286],
        [   53,  4134,   500,   329,   649],
        [   18,    13,  1415, 19707, 22980]])

In [21]:
# 인코딩 결과를 model.generate( )에 투입
generated = model.generate(input['input_ids'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [22]:
# model.generate( ) 결과 건수(len) 확인
len(generated)

3

In [23]:
# 디코딩
generated_text = tokenizer.batch_decode(generated)

# enumerate( )는 변수 generated_text에 있는 키값(value)에
# 0, 1, 2,...로 시작하는 숫자 키(key)를 부여
for i, sentence in enumerate(generated_text):
  print(f'No.{i+1}')
  print(f"{sentence}\n")


No.1
I evaluated the performance of the compiler, the compiler itself is not a compiler.<|endoftext|>

No.2
Vaccine for new grads.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>

No.3
3.14159265 of course<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>



# 20

In [24]:
# 코랩 메모리가 clear된 경우만 transformers 재설치
# !pip install transformers

# 모델 및 토크나이저 불러오기
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

model = AutoModelWithLMHead.from_pretrained("distilgpt2")


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


# 21

In [29]:
input_ids

tensor([[ 40, 588, 308, 457, 780, 340, 338]])

In [32]:
# 토크나이징. 출력은 파이토치 텐서(pt)로 받음
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')

# max-length 값을 12로 설정
# bert 모델이랑 다르게 .generate를 통해 문장을 생성
greedy_output = model.generate(input_ids, max_length=12)

# *표시를 100번 실행하여 *로 구성된 줄 만들기
print("Output:\n" + 100 * '-')

# 인코딩 결과 디코딩
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good thing to have


## 22

In [33]:
# 모델 및 토크나어지 불러오기
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

# 23

In [34]:
# 토크나이징
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')

# max-length 값을 30으로 설정
greedy_output = model.generate(input_ids, max_length=30)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good way to get a feel for the game.


In [46]:
# 보너스 코딩: 다른 예문으로 추가 코딩
# 토크나이징
input_ids = tokenizer.encode("What's your favoriate food?", return_tensors='pt')

# max-length 값을 50으로 설정
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
What's your favoriate food?


In [ ]:
input_ids = tokenizer.encode("Covid19 delta is spreading", return_tensors='pt')

# max-length 값을 50으로 설정
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


# 5장. MLM
- 입력 문서내 토큰의15%를 [MASK] 토큰으로 가리고 원래 단어를 추측하는 빈칸 채우기 작업

------------------
추가로 NSP: 두 개의 문장이 연속적인 문장인지 아닌지를 예측하는 작업  
BERT의 사전학습은 MLM과 NSP를 통해 이루어짐

## 24

In [36]:
# 코랩 메모리가 clear된 경우만 transformers 재설치
!pip install transformers

In [40]:
# transformers 라이브러리에서 pipeline 불로오기
from transformers import pipeline

# pipeline에 과업(fill-mask) 및 모델 지정
# 어떤 작업을 할지
unmasker = pipeline('fill-mask', model='bert-base-uncased')

# pipeline을 인스턴스화한 변수 unmasker에 [MASK] 토큰을 지닌 입력문장 투입
unmasker("MLM and NSP is the [MASK] task of BERT.")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.2572788596153259,
  'token': 2364,
  'token_str': 'main',
  'sequence': 'mlm and nsp is the main task of bert.'},
 {'score': 0.20740669965744019,
  'token': 3078,
  'token_str': 'primary',
  'sequence': 'mlm and nsp is the primary task of bert.'},
 {'score': 0.06773319095373154,
  'token': 2034,
  'token_str': 'first',
  'sequence': 'mlm and nsp is the first task of bert.'},
 {'score': 0.06548505276441574,
  'token': 2430,
  'token_str': 'central',
  'sequence': 'mlm and nsp is the central task of bert.'},
 {'score': 0.061674006283283234,
  'token': 3937,
  'token_str': 'basic',
  'sequence': 'mlm and nsp is the basic task of bert.'}]

# 25

In [38]:
from transformers import pipeline

# 모델명이 바뀌었음에 유의
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker("MLM and NSP is the [MASK] task of BERT.")


[{'score': 0.2590249180793762,
  'token': 3078,
  'token_str': 'primary',
  'sequence': 'mlm and nsp is the primary task of bert.'},
 {'score': 0.1630987823009491,
  'token': 2364,
  'token_str': 'main',
  'sequence': 'mlm and nsp is the main task of bert.'},
 {'score': 0.08182762563228607,
  'token': 4563,
  'token_str': 'core',
  'sequence': 'mlm and nsp is the core task of bert.'},
 {'score': 0.040237750858068466,
  'token': 7037,
  'token_str': 'dual',
  'sequence': 'mlm and nsp is the dual task of bert.'},
 {'score': 0.024844925850629807,
  'token': 4054,
  'token_str': 'principal',
  'sequence': 'mlm and nsp is the principal task of bert.'}]

# 26

In [41]:
from transformers import pipeline

# 모델명이 바뀌었음에 유의
unmasker = pipeline('fill-mask', model='albert-base-v2')
unmasker("mlm and nsp is the [MASK] task of bert.")


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.weight', 'albert.pooler.bias']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.047600988298654556,
  'token': 6612,
  'token_str': 'ultimate',
  'sequence': 'mlm and nsp is the ultimate task of bert.'},
 {'score': 0.024472273886203766,
  'token': 20766,
  'token_str': 'hardest',
  'sequence': 'mlm and nsp is the hardest task of bert.'},
 {'score': 0.023495061323046684,
  'token': 1256,
  'token_str': 'primary',
  'sequence': 'mlm and nsp is the primary task of bert.'},
 {'score': 0.02157498709857464,
  'token': 407,
  'token_str': 'main',
  'sequence': 'mlm and nsp is the main task of bert.'},
 {'score': 0.01808803714811802,
  'token': 18369,
  'token_str': 'foremost',
  'sequence': 'mlm and nsp is the foremost task of bert.'}]